In [26]:
import pandas as pd
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold

In [2]:
from google.colab import files
uploaded = files.upload()

Saving dataset-of-80s_broken.csv to dataset-of-80s_broken.csv


In [4]:
import io
df = pd.read_csv(io.BytesIO(uploaded['dataset-of-80s_broken.csv']))

In [5]:
df['duration_ms'] = df['duration_ms'].apply(lambda x : x/1000)
df['duration_ms'] = df['duration_ms'].round(0)
df['duration_ms'] = df['duration_ms'].apply(lambda x : x / 60)

df.loc[(df['duration_ms'] <= 2), 'duration_ms'] = 0
df.loc[((df['duration_ms'] > 2) & (df['duration_ms'] <= 3)), 'duration_ms'] = 1
df.loc[((df['duration_ms'] > 3) & (df['duration_ms'] <= 4)), 'duration_ms'] = 2
df.loc[((df['duration_ms'] > 4) & (df['duration_ms'] <= 5)), 'duration_ms'] = 3
df.loc[(df['duration_ms'] > 5), 'duration_ms'] = 4

df.loc[(df['sections'] <= 5), 'sections'] = 0
df.loc[((df['sections'] > 5) & (df['sections'] <= 10)), 'sections'] = 1
df.loc[((df['sections'] > 10) & (df['sections'] <= 15)), 'sections'] = 2
df.loc[((df['sections'] > 15) & (df['sections'] <= 20)), 'sections'] = 3
df.loc[(df['sections'] > 20), 'sections'] = 4

In [6]:
df = df.drop(columns=['track', 'artist', 'uri'])

In [8]:
df = df.dropna()

In [ ]:
for column in df.columns.values:
  lower = df[column].quantile(0.25) - 1.5 * (df[column].quantile(0.75) - df[column].quantile(0.25))
  upper = df[column].quantile(0.75) + 1.5 * (df[column].quantile(0.75) - df[column].quantile(0.25))
  df.loc[(df[column] < lower), column] = lower
  df.loc[(df[column] > upper), column] = upper
df

In [45]:


X = df.iloc[:, 0:12].to_numpy()
Y = df['target'].to_numpy()

kfold = KFold(n_splits=10, shuffle=True)

fold_number = 1

accuracies = np.array([])

for train,test in kfold.split(X, Y):
  model = Sequential()
  model.add(Dense(12, input_dim=12, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  opt = keras.optimizers.Adam(learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  print(f'{fold_number} intervalo mokymasis')
  model.fit(X, Y, epochs=30, batch_size=10, verbose=0)
  _, accuracy = model.evaluate(X, Y)
  print('Tikslumas: %.2f' % (accuracy*100))
  print('------------------------------------------------------------------------')
  accuracies = np.append(accuracies, [(accuracy * 100)])
  fold_number = fold_number + 1

print("Vidutinė tikslumo reikšmė po 10 intervalų:  %.2f" % (np.average(accuracies)))



1 intervalo mokymasis
199/199 [==============================] - 0s 868us/step - loss: 0.4838 - accuracy: 0.7743
Tikslumas: 77.43
------------------------------------------------------------------------
2 intervalo mokymasis
199/199 [==============================] - 0s 865us/step - loss: 0.4915 - accuracy: 0.7614
Tikslumas: 76.14
------------------------------------------------------------------------
3 intervalo mokymasis
199/199 [==============================] - 0s 855us/step - loss: 0.4931 - accuracy: 0.7699
Tikslumas: 76.99
------------------------------------------------------------------------
4 intervalo mokymasis
199/199 [==============================] - 0s 906us/step - loss: 0.4823 - accuracy: 0.7704
Tikslumas: 77.04
------------------------------------------------------------------------
5 intervalo mokymasis
199/199 [==============================] - 0s 870us/step - loss: 0.5093 - accuracy: 0.7507
Tikslumas: 75.07
----------------------------------------------------------